# Project 1

Data from: Heyes, Anthony, and Soodeh Saberian. 2019. "Temperature and Decisions: Evidence from 207,000 Court Cases." American Economic Journal: Applied Economics, 11 (2): 238–65.

Notebooks used troughout the code: 
- ISLP-Ch06_varselect_lab.ipynb
- ISLP-TreeModels.ipynb
- CIDP-Chapter_04
- CIDP-Chapter_05
- CIDP-Chapter_07

Code for Project 2 related activities:
- CIBT-11-Propensity-Score
- CIBT-21 Meta-Learners

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import sklearn.linear_model as skl
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import sklearn.model_selection as skm
from sklearn.model_selection import GridSearchCV, KFold
from matplotlib.pyplot import subplots
from statsmodels.discrete.discrete_model import Probit
import statsmodels.api as sm
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import SGDClassifier

!pip install stargazer
from stargazer.stargazer import Stargazer

In [ ]:
#For Tree
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestClassifier as RFC,
      GradientBoostingClassifier as GBC)
from sklearn.metrics import confusion_matrix

In [ ]:
#For Graphs
from scipy import stats

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import graphviz
import networkx as nx
COLORS = [
    '#00B0F0',
    '#FF0000'
]
import matplotlib.patches as patches

In [ ]:
#DAG
!pip install dowhy
import dowhy
from dowhy import CausalModel


In [ ]:
# Propensity Score
from sklearn.linear_model import LogisticRegression
import seaborn as sns


## Data Description 

In [ ]:
#Read the dataset from the replication package
df = pd.read_stata('matched_corrected.dta')
df.describe()

In [ ]:
# Print the list of columns to identify all variables

columns_list = df.columns.tolist()
print(columns_list)

In [ ]:
#Create a dummy for asylum
df['dummy_asylum'] = df['c_asy_type'].apply(lambda x: 1 if x == 'E' else 0)
#Create a dummy for gender
df['dummy_gender'] = df['gender'].apply(lambda x: 1 if x == 'female' else 0)

In [ ]:
#As outlined in the correction article drop the observation for China
df = df[df['nat_name'] != 'CHINA']

In [ ]:
# Get unique values to identify variables for the dummy variables
unique__names = df['nat_name'].unique()
locations = df['location'].unique()

#Create a categorical variable for nationatility

# Define the list of regions
middle_eastern_countries = ["BAHRAIN", "CYPRUS", "EGYPT", "IRAN", "IRAQ", "ISRAEL", "JORDAN", 
    "KUWAIT", "LEBANON", "OMAN", "PALESTINE", "QATAR", "SAUDI ARABIA", 
    "SYRIA", "TURKEY", "UNITED ARAB EMIRATES", "YEMEN"]

africa = ["ERITREA", "RWANDA", "SOMALIA", "SUDAN", "CONGO", "ETHIOPIA", "LIBYA", 
    "MALI", "ANGOLA", "BURUNDI", "TANZANIA", "NIGERIA", "GABON", "GHANA", 
    "SENEGAL", "CHAD", "DJIBOUTI", "CAMEROON", "UGANDA", "KENYA", 
    "ZAMBIA", "MAURITANIA", "SOUTH AFRICA", "GUINEA", "BURKINA FASO", 
    "MOROCCO", "ALGERIA", "COMORO ISLANDS", "EQUATORIAL GUINEA", 
    "CENTRAL AFRICAN REPUBLIC", "CAPE VERDE", "LESOTHO", "SWAZILAND", 
    "GAMBIA", "SIERRA LEONE", "GUINEA BISSAU"]

america = ["GUATEMALA", "EL SALVADOR", "PANAMA", "COLOMBIA", 
    "ARGENTINA", "HAITI", "VENEZUELA", "MEXICO", "CUBA", "DOMINICAN REPUBLIC", 
    "BRAZIL", "CHILE", "SURINAME", "TRINIDAD AND TOBAGO", "JAMAICA", 
    "CANADA", "USA", "ST. KITTS, WEST INDIES", "ANTIGUA AND BARBUDA", 
    "BARBADOS", "BAHAMAS", "BELIZE", "DOMINICA", "GRENADA", 
    "NICARAGUA", "URUGUAY", "PARAGUAY", "ST. LUCIA", "ST. VINCENT AND THE GRENADINES"]

asia = ["PAKISTAN", "VIETNAM", "INDONESIA", "AFGHANISTAN", 
    "IRAN", "BANGLADESH", "PHILIPPINES", "TAIWAN", "MALAYSIA", 
    "KAZAKHSTAN", "KYRGYZSTAN", "THAILAND", "TURKMENISTAN", "UZBEKISTAN", 
    "MONGOLIA", "SRI LANKA", "BHUTAN", "LAOS", "NEPAL", 
    "MYANMAR", "KAMPUCHEA", "BRUNEI", "BURMA", "KOREA", "NORTH KOREA"]

europe = ["RUSSIA", "ARMENIA", "ALBANIA", "YUGOSLAVIA", "UNITED KINGDOM", 
    "BULGARIA", "ROMANIA", "HUNGARY", "POLAND", "CZECH REPUBLIC", 
    "SLOVAK REPUBLIC", "GERMANY", "FRANCE", "ITALY", "SPAIN", 
    "SWEDEN", "DENMARK", "FINLAND", "AUSTRIA", "SWITZERLAND", 
    "BELGIUM", "GREECE", "NETHERLANDS", "CROATIA", "SLOVENIA", 
    "MONACO", "LITHUANIA", "LATVIA", "ESTONIA", "ICELAND"]

# Create the regional variable and set it to 0 by default
df['middleast'] = 0
df['america'] = 0
df['africa'] = 0
df['asia'] = 0
df['europe'] = 0

# Replace with 1 for observations where nationality is in the list of selected regions
df.loc[df['nat_name'].isin(middle_eastern_countries), 'middleast'] = 1
df.loc[df['nat_name'].isin(america), 'america'] = 1
df.loc[df['nat_name'].isin(africa), 'africa'] = 1
df.loc[df['nat_name'].isin(asia), 'asia'] = 1
df.loc[df['nat_name'].isin(europe), 'europe'] = 1

#Create interaction terms
df['middleast_dev'] = df['middleast']*df['temp6t4']
df['america_dev'] = df['america']*df['temp6t4']
df['africa_dev'] = df['africa']*df['temp6t4']
df['asia_dev'] = df['asia']*df['temp6t4']
df['europe_dev'] = df['europe']*df['temp6t4']

In [ ]:
#Create a categorical variable for location and group locations into regions
northeast = ['NEWARK', 'BOSTON', 'NEW YORK CITY', 'BUFFALO', 'PHILADELPHIA', 
    'NEW YORK ANNEX', 'NY DET (VARICK ST.)', 'HARTFORD', 
    '*PA DOC.', 'CLEVELAND', '*BOP  DANBURY', '*RI  DOC',
    '*WISCONSIN DOC', '*NH  DOC', '*SUFFOLK COUNTY','*NEWARK VIDEO HEARINGS','*JESSUP'
    '*BOP ALLENWOOD', '*NORTHERN STATE NJ DOC','YORK COUNTY DET','YORK COUNTY DET']

midwest = ['CHICAGO', 'DETROIT', 'CINCINNATI', 'CLEVELAND', 'ST. LOUIS', 
    'MEMPHIS', 'KANSAS CITY', 'OMAHA', '*MI  DOC', 
    '*IL DOC - STATESVILLE', '*MO DOC', '*OHIO DOC', 
    '*INDIANA YOUTH CENTER']

south = ['ARLINGTON', 'DALLAS', 'HOUSTON', 'MIAMI', 'ATLANTA', 
    'NEW ORLEANS', 'SAN ANTONIO', 'DALLAS DET', 'SAN ANTONIO DET', 
    'HOUSTON DET', 'ATLANTA DET', '*GEORGIA DOC', '*VA DOC', 
    '*DADE COUNTY FL DOC', '*BROWARD  FL DOC', 'ORLANDO', 'KROME DET',
    'PORT ISABEL DET', 'EL PASO', 'EL PASO DET', '*TX DOC', 
    'LOUISVILLE', 'OKLAHOMA CITY', 'OKLAHOMA CITY DET', 
    'BATAVIA SPC', 'BROWARD TRANS CTR','ST. THOMAS', 'ST. CROIX', 'ROLLING PLAINS DETENTION CENTER',
    '*BOP BIG SPRING AIRPARK','BRADENTON DET','SAN ANTONIO DET']

west = ['DENVER', 'SAN DIEGO', 'LOS ANGELES', 'SAN FRANCISCO', 
    'PHOENIX', 'LAS VEGAS', 'RENO', 'SALT LAKE CITY', 'OTAY MESA', 
    'TUCSON', 'HONOLULU', 'SAN JUAN', 'SEATTLE', 'PORTLAND',
    'SAN FRANCISCO DET', 'DENVER DET', 'SAN DIEGO DETAINED', 
    'MIRA LOMA DET', 'HONOLULU DET', '*CO DOC', '*AZ DOC',
    '*WA DOC', '*AK DOC', 'ANCHORAGE', 'SAN PEDRO', 
    'IMPERIAL', '*NM DOC','PORTLAND DET','*MONROE WA DOC','SAN FRANCISCO ANNEX']


# Create the regional variable and set it to 0 by default
df['northeast'] = 0
df['midwest'] = 0
df['south'] = 0
df['west'] = 0


# Replace with 1 for observation where location is in the list of selected regions
df.loc[df['location'].isin(northeast), 'northeast'] = 1
df.loc[df['location'].isin(midwest), 'midwest'] = 1
df.loc[df['location'].isin(south), 'south'] = 1
df.loc[df['location'].isin(west), 'west'] = 1

In [ ]:
#Create a date categorical variable
df['year'] = df['date'].dt.year

#create dummy for year
df['year2000'] = 0
df['year2001'] = 0
df['year2002'] = 0
df['year2003'] = 0
df['year2004'] = 0

# Replace with 1 for observations in a specific year
df.loc[df['year'] == 2000, 'year2000'] = 1
df.loc[df['year'] == 2001, 'year2001'] = 1
df.loc[df['year'] == 2002, 'year2002'] = 1
df.loc[df['year'] == 2003, 'year2003'] = 1
df.loc[df['year'] == 2004, 'year2004'] = 1


# Interaction term for location and year
years = [2000, 2001, 2002, 2003, 2004]
locations = ['northeast', 'midwest', 'south', 'west']

for year in years:
    for location in locations:
        df[f'{location}_year{year}'] = df[location] * df[f'year{year}']

In [ ]:
# Create dummy variables for the months 

df['month'] = df['date'].dt.month
df = pd.get_dummies(df, columns=['month'], prefix='month', drop_first=False)

In [ ]:
#Clean the dataset

#Drop asylum cases with no classification
df = df[df['c_asy_type'].isin(['E', 'I'])]

# Clean dataset by dropping any rows with NA observations
df_final = df.dropna(axis=0) 

In [ ]:
#Calculate the summary statistics for the main variables of interest
summary_stats = df_final[['temp6t4','heat','wind6t4','res','dummy_gender']].describe() 
print(summary_stats)

mean_values = df_final[['temp6t4','heat','wind6t4','res','dummy_gender']].mean()
print(mean_values)

In [ ]:
# Calculate mean
mean_values = df_final[['temp6t4', 'heat', 'wind6t4']].mean()

# Calculate median
median_values = df_final[['temp6t4', 'heat', 'wind6t4']].median()

# Display results
print("Mean values:\n", mean_values)
print("\nMedian values:\n", median_values)

In [ ]:
from matplotlib.ticker import FuncFormatter

# Define the intervals for the histogram (customize these as needed)
bins = [0, 10, 20, 30, 40, 50, 60, 70, 80,90,100]  # Adjust the bin edges according to your data
labels = [f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)]  # Create labels for the intervals

# Create a new column for the binned data
df_final['temp_bins'] = pd.cut(df_final['temp6t4'], bins=bins, labels=labels, right=False)

# Count the occurrences in each bin
counts = df_final['temp_bins'].value_counts().sort_index()

# Plot the histogram
plt.figure(figsize=(7, 6), facecolor='white')  
ax = plt.subplot(111, facecolor='white')  
counts.plot(kind='bar', color='skyblue')
plt.xlabel('Temperature (°F)')
plt.ylabel('Count of Cases')
plt.xticks(rotation=45) 
plt.grid(axis='y')

for spine in ax.spines.values():
    spine.set_visible(False)
    
# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
#Summary statistics per region to check if the observations are balanced

#List of locations
locations = ['northeast', 'midwest', 'south', 'west']

for location in locations:
    # Filter rows where the location is 1 
    df_filtered = df_final[df_final[location] == 1]
    # Calculate summary statistics for resolution in the filtered data
    summary_stats = df_filtered['res'].describe()
    # Print the summary stats for the current location
    print(f"Summary statistics for {location}:")
    print(summary_stats)
    print("\n")  

In [ ]:
# Calculate the average and standard deviation for the dummy_gender variable
mean_value = df_final['dummy_gender'].mean()
std_dev = df_final['dummy_gender'].std()

# Prepare data for the bar plot
summary = pd.DataFrame({
    'Gender': ['Male'],  
    'Mean': [1-mean_value], 
    'Std Dev': [std_dev]  
})

# Create the bar plot
plt.figure(figsize=(7, 6))
plt.figure(figsize=(7, 6), facecolor='white')  
ax = plt.subplot(111, facecolor='white')  
bars = plt.bar(summary['Gender'], summary['Mean'], yerr=summary['Std Dev'], 
                capsize=5, color=['skyblue', 'lightcoral'])

# Add labels and title
plt.ylabel('Proportion')

for spine in ax.spines.values():
    spine.set_visible(False)

# Show the plot
plt.xticks(rotation=0)  
plt.tight_layout()       
plt.show()

## OLS

In [ ]:
#Create my Y variable
Y = np.array(df_final['res'])

In [ ]:
#Create X variables for different specifications
#Note: drop one category for each dummy


#Specification 1
selectedvariables = ['temp6t4','heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

#Specification 2
selectedvariables_noweather = ['temp6t4','chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1','month_2',
                      'month_3','month_4','month_5','month_6','month_7','month_8',
                    'month_9','month_10','month_11']

#Additional Fixed Effects
year_location = ['northeast_year2000', 'northeast_year2001', 'northeast_year2002', 'northeast_year2003', 
                 'northeast_year2004','midwest_year2000', 'midwest_year2001', 'midwest_year2002', 
                 'midwest_year2003', 'midwest_year2004','south_year2000', 'south_year2001', 'south_year2002', 
                 'south_year2003', 'south_year2004','west_year2000', 'west_year2001', 'west_year2002', 
                 'west_year2003']

# Deviation Specification - Future Steps
selectedvariables_deviation = ['deviation','heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

all_variables = selectedvariables + year_location
                        
#Create X variables with different specification
X = df_final[selectedvariables]

X_no_control = df_final[selectedvariables_noweather]

X_all = df_final[all_variables]

X_deviation = df_final[selectedvariables_deviation]

In [ ]:
#Specification 1
model_1 = Probit(Y, X.astype(float))
probit_model1 = model_1.fit()
print(probit_model1.summary())

In [ ]:
# Calculate predicted probabilities
predicted_probs = probit_model1.predict(X.astype(float))

# Calculate marginal effect for the variable of interest
x_temp6t4 = X['temp6t4']  
marginal_effect_temp = probit_model1.params['temp6t4'] * predicted_probs * (1 - predicted_probs)
average_marginal_effect_temp = np.mean(marginal_effect_temp)

print(f"Average Marginal Effect: {average_marginal_effect_temp}")

In [ ]:
# Pass your fitted Probit model to Stargazer
stargazer = Stargazer([probit_model1])

# Output as LaTeX
print(stargazer.render_latex())

In [ ]:
#Specification 1 - Deviation - Future Steps

model_1_deviation = Probit(Y, X_deviation.astype(float))
probit_model1_deviation = model_1_deviation.fit()
print(probit_model1_deviation.summary())

In [ ]:
stargazer_dev = Stargazer([probit_model1_deviation])
# Output as LaTeX
print(stargazer_dev.render_latex())

In [ ]:
#Specification 2

model_2 = Probit(Y, X_no_control.astype(float))
probit_model2 = model_2.fit()
print(probit_model2.summary())

In [ ]:
# Pass your fitted Probit model to Stargazer
stargazer_2 = Stargazer([probit_model2])

# Output as LaTeX
print(stargazer_2.render_latex())

In [ ]:
# Calculate predicted probabilities
predicted_probs = probit_model2.predict(X_no_control.astype(float))

# Calculate marginal effect for the variable of interest 
x_temp6t4 = X['temp6t4']  
marginal_effect_temp = probit_model2.params['temp6t4'] * predicted_probs * (1 - predicted_probs)

# Average marginal effect
average_marginal_effect_temp = np.mean(marginal_effect_temp)

print(f"Average Marginal Effect: {average_marginal_effect_temp}")

In [ ]:
#Specification 3 - Additional Fixed Effects - Not included in the report
model_3 = Probit(Y, X_all.astype(float))
probit_model3 = model_3.fit()
print(probit_model3.summary())

In [ ]:
#Specification - Future Steps - Per region
#Change the df_final['location'] for each region of interest

df_filtered_location = df_final[df_final['west'] == 1]

Y_location = np.array(df_filtered_location['res'])  

#Remove the location fixed effects as the filtered dataset only includes one region
selectedvariables = ['temp6t4','heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

X_location = df_filtered_location[selectedvariables]  

#Run specification 1 with filtered dataset
model_4_location = Probit(Y_location, X_location.astype(float))
probit_model4_location = model_4_location.fit()
print(probit_model4_location.summary())

In [ ]:
stargazer_location = Stargazer([probit_model4_location])
# Output as LaTeX
print(stargazer_location.render_latex())

In [ ]:
# Filter the DataFrame per months (winter vs non. winter months)

df_final['date'] = pd.to_datetime(df_final['date'])

#Filter the dataset to include only months we want (grouping fall with winter and summer and spring)
df_filter_nowinter = df_final[df_final['date'].dt.month.isin([3, 4, 5, 6, 7, 8])]
df_filter_winter = df_final[df_final['date'].dt.month.isin([1,2,9,10,11,12])]

# Define selected variables for no winter, drop winter months dummies
selectedvariables_nowinter = ['skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'temp6t4','deviation', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'heat', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_3',
                     'month_4','month_5','month_6','month_7','month_8']

# Define selected variables for winter, drop non-winter months dummies
selectedvariables_winter = ['skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 
                     'pm_distance', 'temp6t4','deviation', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4', 
                     'rh6t4', 'heat', 'chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1','month_2',
                    'month_9','month_10']


#Create X and Y variables
Y_nowinter = np.array(df_filter_nowinter['res'])  
X_nowinter = df_filter_nowinter[selectedvariables_nowinter]  

Y_winter = np.array(df_filter_winter['res'])  
X_winter = df_filter_winter[selectedvariables_winter]  


In [ ]:
plt.rcParams['font.family'] = 'DejaVu Sans'  

# Calculate the average and standard deviation for winter and non-winter months
winter_mean = df_filter_winter['res'].mean()
winter_std = df_filter_winter['res'].std()
nowinter_mean = df_filter_nowinter['res'].mean()
nowinter_std = df_filter_nowinter['res'].std()

# Prepare data for the bar plot
summary = pd.DataFrame({
    'Season': ['Winter and Fall', 'Summer and Spring'],
    'Mean': [winter_mean, nowinter_mean],
    'Std Dev': [winter_std, nowinter_std]
})

print(winter_mean)
print(winter_std)
print(nowinter_mean)
print(nowinter_std)

# Create the bar plot with error bars
plt.figure(figsize=(7, 7), facecolor='white')
ax = plt.subplot(111, facecolor='white')
ax.grid(False)  # Turn off gridlines


# Plot bars with error bars
bars = plt.bar(summary['Season'], summary['Mean'], yerr=summary['Std Dev'], capsize=5, color=['skyblue', 'lightcoral'])

# Add labels and title
plt.ylabel('Average Resolution')

# Remove plot spines for a clean look
for spine in ax.spines.values():
    spine.set_visible(False)

# Show the plot
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Model with dataframe filtered with no winter months - Future Steps

model_5 = Probit(Y_nowinter, X_nowinter.astype(float))
probit_model5 = model_5.fit()
print(probit_model5.summary())

In [ ]:
stargazer_non_winter = Stargazer([probit_model5])
# Output as LaTeX
print(stargazer_non_winter.render_latex())

In [ ]:
# Model with dataframe filtered with only winter months - Future Steps

model_6 = Probit(Y_winter, X_winter.astype(float))
probit_model6 = model_6.fit()
print(probit_model6.summary())

In [ ]:
stargazer_winter = Stargazer([probit_model6])
# Output as LaTeX
print(stargazer_winter.render_latex())

The most interesting finding is that temperature has a positive coefficient in our logit model (opposite to the original paper) and almost 0 in the Ridge model. Meanwhile, in the Lasso model temperature has a negative coefficient in line with the original paper. Perhaps when more variables are included the effect of temperature is diminished. 

## Running Ridge 

In [ ]:
new_names = {
    'temp6t4': 'Temperature',
    'middleast_dev': 'Middle East*Temperature',
    'america_dev': 'America*Temperature',
    'africa_dev': 'Africa*Temperature',
    'europe_dev': 'Europe*Temperature'
}

# Rename the columns in df_final
df_final_renamed = df_final.rename(columns=new_names)

# Now select the renamed variables for X_Ridge
selected_variables_nofe = ['Temperature', 'Middle East*Temperature', 'America*Temperature', 'Africa*Temperature', 'Europe*Temperature']
X_Ridge = df_final_renamed[selected_variables_nofe]

In [ ]:
#Run a Ridge Model
#Code in this section based on the notebook: ISLP-Ch06_varselect_lab.ipynb

coefficients = []
#Calculate lambda from 10^8 to 10^-2
lambdas = 10**np.linspace(8, -2, 100) / Y.std()
scaler = StandardScaler(with_mean=True,  with_std=True)
for lam in lambdas:
    ridge = SGDClassifier(loss='log_loss', penalty='l2',alpha=lam)
    # Create a pipeline with scaling and the classifier
    pipe = Pipeline(steps=[('scaler', scaler), ('ridge', ridge)])
    # Fit the pipeline to the data
    pipe.fit(X_Ridge, Y)
    # Store the coefficients 
    coefficients.append(pipe.named_steps['ridge'].coef_.flatten())

#solution containing all our coefficients    
soln_array = np.array(coefficients)

# Create a DataFrame with the solution path, for easy  transposing soln_array so features are in columns
soln_path = pd.DataFrame(soln_array, columns=X_Ridge.columns, index=-np.log(lambdas))
# Name the index to indicate it's the negative log of lambda
soln_path.index.name = 'negative log(lambda)'

In [ ]:
#Plot the graph  
path_fig, ax = subplots(figsize=(8,8))
soln_path.plot(ax=ax, legend=False)
ax.set_xlabel('$-\log(\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
ax.legend(loc='upper left');

path_fig.patch.set_facecolor('white')  
ax.set_facecolor('white')

In [ ]:
# Initialize K-Fold cross-validation strategy
kfold = KFold(n_splits=5)

param_grid = {'ridge__alpha': lambdas}

# Use GridSearchCV to find the best alpha using cross-validation
#Since we have a categorical variable the scoring is accuracy (where it defines how accurate we predict y)
grid_search = GridSearchCV(pipe, param_grid, cv=kfold, scoring='accuracy', return_train_score=True)

# Fit the model using cross-validation to find the best alpha
grid_search.fit(X_Ridge, Y)

# Find optimal lambda
tuned_ridge = grid_search.best_estimator_.named_steps['ridge']

# Retrieve the mean and standard deviation of cross-validation scores
mean_scores = grid_search.cv_results_['mean_test_score']
std_scores = grid_search.cv_results_['std_test_score']

In [ ]:
# Plotting the Cross-Validated Accuracy and Error Bars
ridgeCV_fig, ax = subplots(figsize=(8, 8))
ax.errorbar(-np.log(lambdas), mean_scores, yerr=std_scores / np.sqrt(kfold.get_n_splits()), fmt='o')
ax.axvline(-np.log(grid_search.best_params_['ridge__alpha']), c='k', ls='--')
ax.set_xlabel('$-\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
ax.set_title('Cross-Validation Accuracy with Error Bars')

In [ ]:
coefficients = tuned_ridge.coef_.flatten()  
variable_names = X_Ridge.columns  # Get the names of the features

# Create a dictionary mapping variable names to their coefficients
coef_mapping = {variable: coef for variable, coef in zip(variable_names, coefficients)}

# Convert the dictionary to a DataFrame for better visualization
coef_df = pd.DataFrame(list(coef_mapping.items()), columns=['Variable', 'Coefficient'])
print(coef_df)

## Running Lasso

In [ ]:
#Code in this section based on the notebook: ISLP-Ch06_varselect_lab.ipynb

#Running Lasso
coefficients_l = []
for lam in lambdas:
    lasso = SGDClassifier(loss='log_loss', penalty='l1',alpha=lam)
    # Create a pipeline with scaling and the classifier
    pipe_l = Pipeline(steps=[('scaler', scaler), ('lasso', lasso)])
    # Fit the pipeline to the data
    pipe_l.fit(X, Y)
    # Store the coefficients 
    coefficients_l.append(pipe_l.named_steps['lasso'].coef_.flatten())

soln_array_l = np.array(coefficients_l)
# Create a DataFrame with the solution path, transposing soln_array so features are in columns
soln_path_l = pd.DataFrame(soln_array_l, columns=X.columns, index=-np.log(lambdas))
# Name the index to indicate it's the negative log of lambda
soln_path_l.index.name = 'negative log(lambda)'

In [ ]:
path_fig_l, ax = subplots(figsize=(8,8))
soln_path_l.plot(ax=ax, legend=False)
ax.set_xlabel('$-\log(\lambda)$', fontsize=20)
ax.set_ylabel('Standardized coefficients', fontsize=20)
ax.legend(loc='upper left');

In [ ]:
kfold = KFold(n_splits=5)

# Define a grid of alpha values to search over
param_grid_l = {'lasso__alpha': lambdas}

# Use GridSearchCV to find the best alpha using cross-validation
# Since we have a categorical variable the scoring is not MSE but accuracy
grid_search_l = GridSearchCV(pipe_l, param_grid_l, cv=kfold, scoring='accuracy', return_train_score=True)

# Fit the model using cross-validation to find the best alpha
grid_search_l.fit(X, Y)

# Get the tuned LASSO model (SGDClassifier)
tuned_lasso = grid_search_l.best_estimator_.named_steps['lasso']

# Retrieve the mean and standard deviation of cross-validation scores
mean_scores_l = grid_search_l.cv_results_['mean_test_score']
std_scores_l = grid_search_l.cv_results_['std_test_score']

In [ ]:
# Plotting the Cross-Validated Accuracy and Error Bars
lassoCV_fig, ax = subplots(figsize=(8, 8))
ax.errorbar(-np.log(lambdas), mean_scores_l, yerr=std_scores_l / np.sqrt(kfold.get_n_splits()), fmt='o')
ax.axvline(-np.log(grid_search_l.best_params_['lasso__alpha']), c='k', ls='--')
ax.set_xlabel('$-\log(\\lambda)$', fontsize=20)
ax.set_ylabel('Cross-validated Accuracy', fontsize=20)
ax.set_title('Cross-Validation Accuracy with Error Bars')

In [ ]:
coefficients_l = tuned_lasso.coef_.flatten()  
variable_names = X.columns  # Get the names of the features

# Create a dictionary mapping variable names to their coefficients
coef_mapping_l = {variable: coef for variable, coef in zip(variable_names, coefficients_l)}

# Convert the dictionary to a DataFrame for better visualization
coef_df = pd.DataFrame(list(coef_mapping_l.items()), columns=['Variable', 'Coefficient'])
print(coef_df)

## Regresion Trees 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

clf = DTC(criterion='entropy', 
          max_depth = 3,
          random_state=0)   

clf.fit(X, Y)

In [ ]:
#Identify columns
X.columns

In [ ]:
#Rename columns for better labelling of trees
X_detailed = ['Average temperature', 'Heat index', 'Sky coverage', 'Carbon monoxide levels', 
              'Distance CO source', 'Ozone levels', 'Distance ozone source', 'PM levels', 
              'Distance PM source', 'Atmospheric pressure', 'Dew point temperature', 'Precipitation', 
              'Wind speed', 'Relative humidity', 'Judge identifier', 'Asylum application', 
              'Gender', 'Middle Eastern', 'American', 'African', 'European', 'Northeast', 'Midwest', 'South', 
              '2000','2001', '2002', '2003', 'Interaction of temperature and Middle Eastern',
              'Interaction of temperature and American', 'Interaction of temperature and African', 
              'Interaction of temperature and European', 'January', 'February', 'March', 'April', 'May', 'June', 
              'July', 'August', 'September', 'October', 'November']

In [ ]:
feature_names = X_detailed

In [ ]:
accuracy_score(Y, clf.predict(X)) 

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

# Remove the gray background
fig.patch.set_facecolor('white')  
ax.set_facecolor('white')        

plot_tree(clf,
          feature_names=feature_names,
          ax=ax,
          filled=True,  
          rounded=True,  
          fontsize=7,   
          proportion=True)  

# Show plot
plt.show()

In [ ]:
#Cross validation 
validation = skm.ShuffleSplit(n_splits=1,
                              test_size=200,
                              random_state=0)
results = skm.cross_validate(clf,
                             X,
                             Y,
                             cv=validation)
results['test_score']

In [ ]:
#Split dataset
(X_train,
 X_test,
 Y_train,
 Y_test) = skm.train_test_split(X,
                                   Y,
                                   test_size=0.5,
                                   random_state=0)

In [ ]:
clf = DTC(criterion='entropy', random_state=0)
clf.fit(X_train, Y_train)
accuracy_score(Y_test, clf.predict(X_test))

In [ ]:
ccp_path = clf.cost_complexity_pruning_path(X_train, Y_train)
kfold = skm.KFold(5,
                  random_state=1,
                  shuffle=True)

## Bagging 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

bag_temperature = RFC(max_features=X_train.shape[1], random_state=0)
bag_temperature.fit(X_train, Y_train)

In [ ]:
bag_temperature = RFC(max_features=X_train.shape[1],
                n_estimators=500,#how many trees you are running
                random_state=0).fit(X_train, Y_train)
y_hat_bag = bag_temperature.predict(X_test)
accuracy_bagging = accuracy_score(Y_test, y_hat_bag)
accuracy_bagging

In [ ]:
feature_imp_bag = pd.DataFrame(
    {'importance':bag_temperature.feature_importances_},
    index=feature_names)
feature_imp_bag.sort_values(by='importance', ascending=False)

In [ ]:
feature_imp_bag = pd.DataFrame(
    {'importance': bag_temperature.feature_importances_},
    index=feature_names
)

# Sort the feature importances
feature_imp_bag = feature_imp_bag.sort_values(by='importance', ascending=False)

# Plot the feature importances
plt.figure(figsize=(10, 6))
plt.barh(feature_imp_bag.index, feature_imp_bag['importance'], color='skyblue')
plt.xlabel('Importance')
plt.title('Feature Importances (Bagged Model)')
plt.gca().invert_yaxis()  # To display the most important feature at the top
plt.show()

## Random Forests 

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

RF_temperature = RFC(max_features=6,
               random_state=0).fit(X_train, Y_train)
y_hat_RF = RF_temperature.predict(X_test)
accuracy_RF = accuracy_score(Y_test, y_hat_RF)
accuracy_RF

In [ ]:
feature_imp = pd.DataFrame(
    {'importance':RF_temperature.feature_importances_},
    index=feature_names)
feature_imp.sort_values(by='importance', ascending=False)

## Boosting

In [ ]:
#Code in this section based on the notebook: ISLP-TreeModels.ipynb

boost_temperature = GBC(n_estimators=500,
                   learning_rate=0.001,
                    max_depth = 3,
                   random_state=0)
boost_temperature.fit(X_train, Y_train)

In [ ]:
boost_temperature = GBC(n_estimators=500,
                   learning_rate=0.001,
                    max_depth = 3,
                   random_state=0)
boost_temperature.fit(X_train,Y_train)
y_hat_boost = boost_temperature.predict(X_test);
accuracy_boosting = accuracy_score(Y_test, y_hat_boost)
accuracy_boosting

In [ ]:
feature_imp = pd.DataFrame(
    {'importance':boost_temperature.feature_importances_},
    index=feature_names)
feature_imp.sort_values(by='importance', ascending=False)

## Directed Acyclic Graph (DAG) and Causal Relationship 

In [ ]:
#Code in this section based on the notebook: CIDP-Chapter_04

# Define the graph
sample_gml = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 5
    target 4
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 4
    ]
edge [
    source 5
    target 2
    ]
]
    
    """

In [ ]:
# Get the graph
graph = nx.parse_gml(sample_gml)

# Plot
nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
# Define the graph
sample_gml2 = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 5
    target 4
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 1
    ]
edge [
    source 5
    target 2
    ]
]
    
    """

In [ ]:
# Get the graph
graph = nx.parse_gml(sample_gml2)

# Plot
nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
# Define the graph
sample_gml3 = """graph [
directed 1

node [
    id 1
    label "Cognitive"
    ]
    
node [
    id 2
    label "Weather"
    ]

node [
    id 4
    label "Resolution"
    ]
    
node [
    id 5
    label "Location"
    ]

node [
    id 6
    label "Judge"
    ]

node [
    id 7
    label "Nationality"
    ]
    
edge [
    source 2
    target 1
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 1
    target 6
    ]
edge [
    source 7
    target 1
    ]
edge [
    source 5
    target 2
    ]
edge [
    source 5
    target 6
    ]
]
    
    """

In [ ]:
# Get the graph
graph = nx.parse_gml(sample_gml3)

# Plot
nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

In [ ]:
# Define the graph
gml_final = """graph [
directed 1
    
node [
    id 1
    label "midwest"
    ]

node [
    id 2
    label "deviation"
    ]

node [
    id 4
    label "res"
    ]
    
node [
    id 5
    label "northeast"
    ]

node [
    id 6
    label "chair"
    ]

node [
    id 7
    label "america"
    ]
node [
    id 8
    label "south"
    ]
node [
    id 9
    label "west"
    ]
node [
    id 11
    label "cognitive"
    ]
    
edge [
    source 2
    target 11
    ]

edge [
    source 6
    target 4
    ]

edge [
    source 7
    target 11
    ]
edge [
    source 5
    target 2
    ]
edge [
    source 1
    target 2
    ]
edge [
    source 8
    target 2
    ]
edge [
    source 9
    target 2
    ]
edge [
    source 5
    target 11
    ]
edge [
    source 1
    target 11
    ]
edge [
    source 8
    target 11
    ]
edge [
    source 9
    target 11
    ]
edge [ 
    source 11
    target 6
    ]
]

    
    """

In [ ]:
# Get the graph
graph = nx.parse_gml(gml_final)

# Plot
nx.draw(
    G=graph, 
    with_labels=True,
    node_size=2500,
    node_color=COLORS[0],
    font_color='black',
    font_size = 8
)

## Test DAG

In [ ]:
#Code in this section based on the notebook:  CIDP-Chapter_07

#Model the problem
model = CausalModel(
data=df_final,
treatment=['deviation'],
outcome="res",
graph=gml_final)

In [ ]:
#Identify the estimand
estimand = model.identify_effect()

In [ ]:
print(estimand)

In [ ]:
#obtain estimates
estimate = model.estimate_effect(
identified_estimand=estimand,
method_name="backdoor.linear_regression")

In [ ]:
print(estimate)

In [ ]:
#perform refutation test
#Refutation test on whether estimate is influenced by unobserved confounders = random_common_cause 
refute_subset = model.refute_estimate(
estimand=estimand,
estimate=estimate,
method_name="random_common_cause",
subset_fraction=0.4)

In [ ]:
print(refute_subset)
#High p-value suuggests that the random common cause does not have a meaningful impact on the relationship between 
#temperature and the outcome, providing confidence in the stability of findings.

## IPW

In [ ]:
#Code in this section based on the notebook: CIBT-11-Propensity-Score

#Heat stress begins at 33 degrees
df_final['T_binary'] = (df_final['temp6t4'] > 75).astype(int)
print(df_final['T_binary'].value_counts())

In [ ]:
T = 'T_binary'
Y = 'res'
X = ['chair', 'dummy_asylum', 'dummy_gender', 
                     'middleast', 'america', 'africa', 'europe', 'northeast', 'midwest', 
                     'south', 'year2000', 'year2001', 'year2002', 
                     'year2003','middleast_dev','america_dev','africa_dev','europe_dev','month_1',
                     'month_2','month_3','month_4','month_5','month_6','month_7','month_8',
                     'month_9','month_10','month_11']

#'heat','skycover', 'co', 'co_distance', 'ozone', 'ozone_distance', 'pm', 'pm_distance', 'press6t4', 'dew6t4', 'prcp6t4', 'wind6t4','rh6t4', 

df_final_limited = df_final[(df_final['temp6t4'] > 60) & (df_final['temp6t4'] < 90)]

ps_model = LogisticRegression(C=1e6).fit(df_final_limited[X], df_final_limited[T])

data_ps = df_final_limited.assign(propensity_score=ps_model.predict_proba(df_final_limited[X])[:, 1])

data_ps[["T_binary", "res", "propensity_score"]].head()

In [ ]:
weight_t = 1/data_ps.query("T_binary==1")["propensity_score"]
weight_nt = 1/(1-data_ps.query("T_binary==0")["propensity_score"])
print("Original Sample Size", df.shape[0])
print("Treated Population Sample Size", sum(weight_t))
print("Untreated Population Sample Size", sum(weight_nt))

In [ ]:
sns.distplot(data_ps.query("T_binary==0")["propensity_score"], kde=False, label="Non Treated")
sns.distplot(data_ps.query("T_binary==1")["propensity_score"], kde=False, label="Treated")
plt.title("Positivity Check")
plt.legend();

In [ ]:
# Remove observations with propensity score = 1
data_ps = data_ps[data_ps["propensity_score"] < 1]


# Calculate weights for treated and control groups
treated_data = data_ps.query("T_binary == 1")
control_data = data_ps.query("T_binary == 0")

# Calculate weighted outcomes for treated (Y1) and control (Y0)
y1 = sum(treated_data["T_binary"] * weight_t) / len(treated_data)
y0 = sum(control_data["T_binary"] * weight_nt) / len(control_data)

# Calculate the ATE (Average Treatment Effect)
ate = np.mean(weight_t * treated_data["T_binary"]) - np.mean(weight_nt * control_data["T_binary"])

# Print results
print("Y1:", y1)
print("Y0:", y0)
print("ATE:", ate)


In [ ]:
# Calculate weights for treated and control groups
treated_data = data_ps.query("res == 1")
control_data = data_ps.query("res == 0")

if not treated_data.empty:
    weight_t = 1 / treated_data["propensity_score"]
    print("Weight_t:", weight_t)

if not control_data.empty:
    weight_nt = 1 / (1 - control_data["propensity_score"])
    print("Weight_nt:", weight_nt)


In [ ]:
print(data_ps["propensity_score"].min(), data_ps["propensity_score"].max())


In [ ]:
def run_ps(df_final, X, T, y):
    # estimate the propensity score
    ps = LogisticRegression(C=1e6, max_iter=2000, solver='liblinear').fit(df_final[X], df_final[T]).predict_proba(df_final[X])[:, 1]
    weight = (df_final[T]-ps) / (ps*(1-ps)) # define the weights
    return np.mean(weight * df_final[y]) # compute the ATE

sample_df = df_final.sample(frac=1, replace=True)
ate = run_ps(sample_df, X, T, Y)
print(ate)

In [ ]:
from joblib import Parallel, delayed # for parallel processing

sample_df = df_final.sample(frac=0.5, replace=True)

# define function that computes the IPTW estimator
def run_ps(sample_df, X, T, y):
    # estimate the propensity score
    ps = LogisticRegression(C=1e6, max_iter=2000, solver='liblinear').fit(sample_df[X], sample_df[T]).predict_proba(sample_df[X])[:, 1]
    weight = (sample_df[T]-ps) / (ps*(1-ps)) # define the weights
    return np.mean(weight * df_final[y]) # compute the ATE

np.random.seed(88)
# run 100 bootstrap samples
bootstrap_sample = 100
ates = Parallel(n_jobs=4)(delayed(run_ps)(df_final.sample(frac=1, replace=True), X, T, Y)
                          for _ in range(bootstrap_sample))
ates = np.array(ates)

## Learners 

In [ ]:
#Code in this section based on the notebook: CIBT-21 Meta-Learners
!pip install lightgbm

In [ ]:
from sklearn.model_selection import train_test_split

# Split df_final randomly into two parts (e.g., 50% - 50%)
df_train, df_test = train_test_split(df_final, test_size=0.5, random_state=42)

In [ ]:
from lightgbm import LGBMRegressor

np.random.seed(123)
s_learner = LGBMRegressor(max_depth=3, min_child_samples=30)
s_learner.fit(df_train[X+[T]], df_train[Y]);

In [ ]:
s_learner_cate_train = (s_learner.predict(df_train[X].assign(**{T: 1})) -
                        s_learner.predict(df_train[X].assign(**{T: 0})))

s_learner_cate_test = df_test.assign(
    cate=(s_learner.predict(df_test[X].assign(**{T: 1})) - # predict under treatment
          s_learner.predict(df_test[X].assign(**{T: 0}))) # predict under control
)

In [ ]:
def elast(data, y, t):
    return (np.sum((data[t] - data[t].mean())*(data[y] - data[y].mean())) /
            np.sum((data[t] - data[t].mean())**2))

def cumulative_gain(dataset, prediction, y, t, min_periods=30, steps=100):
    size = dataset.shape[0]
    ordered_df = dataset.sort_values(prediction, ascending=False).reset_index(drop=True)
    n_rows = list(range(min_periods, size, size // steps)) + [size]
    return np.array([elast(ordered_df.head(rows), y, t) * (rows/size) for rows in n_rows])

In [ ]:
gain_curve_test = cumulative_gain(s_learner_cate_test, "cate", y="res", t="T_binary")
gain_curve_train = cumulative_gain(df_train.assign(cate=s_learner_cate_train), "cate",y="res", t="T_binary")
plt.plot(gain_curve_test, color="C0", label="Test")
plt.plot(gain_curve_train, color="C1", label="Train")
plt.plot([0, 100], [0, elast(df_test, y="res", t="T_binary")], linestyle="--", color="black", label="Baseline")
plt.legend()
plt.title("S-Learner");

In [ ]:
np.random.seed(123)

m0 = LGBMRegressor(max_depth=2, min_child_samples=60)
m1 = LGBMRegressor(max_depth=2, min_child_samples=60)

m0.fit(df_train.query(f"{T}==0")[X], df_train.query(f"{T}==0")[Y])
m1.fit(df_train.query(f"{T}==1")[X], df_train.query(f"{T}==1")[Y])

# estimate the CATE
t_learner_cate_train = m1.predict(df_train[X]) - m0.predict(df_train[X])
t_learner_cate_test = df_test.assign(cate=m1.predict(df_test[X]) - m0.predict(df_test[X]))

In [ ]:
gain_curve_test = cumulative_gain(t_learner_cate_test, "cate", y="res", t="T_binary")
gain_curve_train = cumulative_gain(df_train.assign(cate=t_learner_cate_train), "cate", y="res", t="T_binary")
plt.plot(gain_curve_test, color="C0", label="Test")
plt.plot(gain_curve_train, color="C1", label="Train")
plt.plot([0, 100], [0, elast(df_test, "res", "T_binary")], linestyle="--", color="black", label="Baseline")
plt.legend();
plt.title("T-Learner");

In [ ]:
np.random.seed(123)

# first stage models
m0 = LGBMRegressor(max_depth=2, min_child_samples=30)
m1 = LGBMRegressor(max_depth=2, min_child_samples=30)

# propensity score model
g = LogisticRegression(solver="lbfgs", penalty='none') 

m0.fit(df_train.query(f"{T}==0")[X], df_train.query(f"{T}==0")[Y])
m1.fit(df_train.query(f"{T}==1")[X], df_train.query(f"{T}==1")[Y])
                       
g.fit(df_train[X], df_train[T]);

In [ ]:
d_train = np.where(df_train[T]==0,
                   m1.predict(df_train[X]) - df_train[Y],
                   df_train[Y] - m0.predict(df_train[X]))

# second stage
mx0 = LGBMRegressor(max_depth=2, min_child_samples=30)
mx1 = LGBMRegressor(max_depth=2, min_child_samples=30)

mx0.fit(df_train.query(f"{T}==0")[X], d_train[df_train[T]==0])
mx1.fit(df_train.query(f"{T}==1")[X], d_train[df_train[T]==1]);

In [ ]:
def ps_predict(df, t): 
    return g.predict_proba(df[X])[:, t]
    
    
x_cate_train = (ps_predict(df_train,1)*mx0.predict(df_train[X]) +
                ps_predict(df_train,0)*mx1.predict(df_train[X]))

x_cate_test = df_test.assign(cate=(ps_predict(df_test,1)*mx0.predict(df_test[X]) +
                                ps_predict(df_test,0)*mx1.predict(df_test[X])))

In [ ]:
gain_curve_test = cumulative_gain(x_cate_test, "cate", y="res", t="T_binary")
gain_curve_train = cumulative_gain(df_train.assign(cate=x_cate_train), "cate", y="res", t="T_binary")
plt.plot(gain_curve_test, color="C0", label="Test")
plt.plot(gain_curve_train, color="C1", label="Train")
plt.plot([0, 100], [0, elast(df_test, "res", "T_binary")], linestyle="--", color="black", label="Baseline")
plt.legend();
plt.title("X-Learner");